In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import re
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import save_model

In [4]:
physical_devices = tf.config.list_physical_devices("GPU")
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
data_path = pathlib.Path("../data/raw/PlantVillage")

In [6]:
features = np.load("features.npy")
labels = np.load("labels.npy")

In [7]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [8]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

In [9]:
train_features.shape

(43444, 128, 128, 3)

In [10]:
train_labels.shape

(43444, 2)

In [11]:
test_features.shape

(10861, 128, 128, 3)

In [12]:
test_labels.shape

(10861, 2)

In [13]:
input_shape=(128,128,3)
classes = 2
epoch = 10
batch_size = 64

resnet50 = ResNet50(weights=None, input_shape=input_shape, classes=classes)

# resnet50.summary()

resnet50.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

early_stop= EarlyStopping(monitor='val_loss',patience=2)

mcp_save = ModelCheckpoint('mdl_resnet50_1.hdf5', save_best_only=True, monitor='val_loss', mode='min')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

resnet50.fit(train_features, train_labels,
          epochs=epoch,
          batch_size=batch_size,
          validation_data=(test_features,test_labels), 
          callbacks=[early_stop, learning_rate_reduction])

Epoch 1/10
679/679 [==============================] - 125s 132ms/step - loss: 0.4640 - accuracy: 0.8611 - val_loss: 0.2954 - val_accuracy: 0.8945
Epoch 2/10
679/679 [==============================] - 88s 130ms/step - loss: 0.1308 - accuracy: 0.9512 - val_loss: 0.2350 - val_accuracy: 0.9257
Epoch 3/10
679/679 [==============================] - 88s 130ms/step - loss: 0.0744 - accuracy: 0.9723 - val_loss: 0.4533 - val_accuracy: 0.8719
Epoch 4/10
679/679 [==============================] - 88s 130ms/step - loss: 0.0574 - accuracy: 0.9800 - val_loss: 0.1227 - val_accuracy: 0.9576
Epoch 5/10
679/679 [==============================] - 88s 130ms/step - loss: 0.0423 - accuracy: 0.9855 - val_loss: 10.5637 - val_accuracy: 0.7287
Epoch 6/10
679/679 [==============================] - 88s 130ms/step - loss: 0.0526 - accuracy: 0.9805 - val_loss: 0.4614 - val_accuracy: 0.8360


In [14]:
filepath = '../models/resnet50'
save_model(resnet50, filepath)

metrics=pd.DataFrame(resnet50.history.history)
metrics

INFO:tensorflow:Assets written to: ../models/resnet50/assets


,loss,accuracy,val_loss,val_accuracy,lr
0,0.288530,0.896879,0.295356,0.894485,0.001
1,0.116333,0.956473,0.235050,0.925697,0.001
2,0.070878,0.973966,0.453324,0.871927,0.001
3,0.058224,0.979445,0.122701,0.957555,0.001
4,0.054210,0.980918,10.563708,0.728662,0.001
5,0.046593,0.982875,0.461434,0.836019,0.001


In [15]:
results = resnet50.evaluate(test_features,test_labels)
print(f"test acc = {results[1]}\ntest loss = {results[0]}")

340/340 [==============================] - 9s 25ms/step - loss: 0.4614 - accuracy: 0.8360
test acc = 0.8360188007354736
test loss = 0.4614337980747223
